# Sliding epochs

In [ ]:
from gtheory import config as cf
from glob import glob
from gtheory.utils.misc import sort_path
import mne
from mne.preprocessing import compute_current_source_density
from mne import read_epochs
import matplotlib.pyplot as plt
pt = cf.root_path ()

In [ ]:
sbj1_path=sort_path(glob ( f"{pt['dir_root']}/*/*/*.fif" ))[0]
epochs=read_epochs ( sbj1_path, preload=True )
epochs.set_eeg_reference(projection=True).apply_proj()
lambda2=1e-5
stiffness=4
epochs_csd=compute_current_source_density ( epochs, lambda2=lambda2, stiffness=stiffness )

epoch_set=epochs_csd # shallow copy to respect def name

Slide the epochs with windows size `overlap_window`

In [ ]:
import numpy as np
duration=2 # in seconds

ep_ls = []
for epochs_data, t in zip ( epoch_set, epoch_set.events ):
    ep = mne.make_fixed_length_epochs ( mne.io.RawArray ( epochs_data, epoch_set.info ),
                                            duration=duration, preload=True, overlap=0 )
    ep.events [:, -1] = np.arange ( 0, len ( ep ) ) + (251 if t [-1] == 1 else 252 * 10)
    ep.info ['description'] = str ( t )
    n=ep.events [:, -1].tolist()
    ep.event_id = dict(zip(map(str,n),n))
    ep_ls.append ( ep )

Lets visualise the epoch arrangement

In [ ]:
epoch_set[0].plot(n_channels=2);

In [ ]:
ep_ls[0].plot(n_channels=2);

# sliding windows
Let make an overlap sliding windows

In [ ]:
duration=4 # in seconds
overlap=2
ep_ls_overlap = []
for epochs_data, t in zip ( epoch_set, epoch_set.events ):
    ep = mne.make_fixed_length_epochs ( mne.io.RawArray ( epochs_data, epoch_set.info ),
                                            duration=duration, preload=True, overlap=overlap )
    ep.events [:, -1] = np.arange ( 0, len ( ep ) ) + (251 if t [-1] == 1 else 252 * 10)
    ep.info ['description'] = str ( t )
    n=ep.events [:, -1].tolist()
    ep.event_id = dict(zip(map(str,n),n))
    ep_ls_overlap.append ( ep )

Lets visualise the epoch arrangment

In [ ]:
epoch_set[0].plot(n_channels=2,title='Original Epoch');

In [ ]:
ep_ls_overlap[0].plot(n_channels=2,title='Overlap windows',show=False);